In [2]:
from src.models import Message, ConversationEntry
from src.openai import DailyAgent
from src.logger import get_logger
from datetime import datetime
from dotenv import load_dotenv
from src.mongo import MongoManager
import os
import uuid

# Initialize logger
logger = get_logger(name=None, log_level="INFO")

# Load environment variables from a .env file
load_dotenv()
logger.info("Environment variables loaded")

# Generate session ID
session_id = str(uuid.uuid4())
logger.info(f"Generated session ID: {session_id}")

# Define the chat history
chat_history = [
    Message(role="system", content="You are a helpful assistant."),
    Message(role="user", content="Hello, how are you?"),
]
logger.debug(f"Chat history initialized with {len(chat_history)} messages")

agent = DailyAgent()
logger.info("DailyAgent initialized")

try:
    # Generate a response from the assistant
    response = agent.create(chat_history=chat_history)
    logger.info("Successfully generated response from assistant")

    # Print the chat history including the assistant's response
    for message in response:
        print(f"{message.role}: {message.content}")
        logger.debug(f"Message from {message.role}: {message.content}")

    # Create conversation entry and store in MongoDB
    mongo_manager = MongoManager()
    conversation = ConversationEntry(
        session_id=session_id,
        user_id="default_user",
        messages=response,
        text_content=" ".join([msg.content for msg in response])
    )
    try:
        mongo_manager.create_conversation(conversation)
        logger.info("Created new conversation in MongoDB")
    except Exception as e:
        if "duplicate key error" in str(e).lower():
            mongo_manager.update_conversation(conversation)
            logger.info("Updated existing conversation in MongoDB")
        else:
            logger.error(f"MongoDB operation failed: {str(e)}")
            raise
    logger.info("Conversation stored in MongoDB")

except Exception as e:
    logger.exception("Error occurred while generating response")
    raise

2024-12-16 19:49:44 - root - INFO - Environment variables loaded
2024-12-16 19:49:44 - root - INFO - Generated session ID: e701deb6-4f15-4bfb-8028-c6d4de442848


2024-12-16 19:49:44 - root - INFO - DailyAgent initialized
2024-12-16 19:49:44 - root - INFO - Using model: gpt-4
2024-12-16 19:49:44 - root - INFO - Generating response from OpenAI
2024-12-16 19:49:46 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-12-16 19:49:46 - root - INFO - Assistant response added to chat history
2024-12-16 19:49:46 - root - INFO - Successfully generated response from assistant
system: You are a helpful assistant.
user: Hello, how are you?
assistant: Hello! As an artificial intelligence, I don't have feelings, but I'm here and ready to assist you. How can I help you today?
2024-12-16 19:49:46 - root - INFO - Successfully connected to MongoDB database: medassistant
2024-12-16 19:49:46 - root - INFO - Created new conversation with ID: 6760764a41ebd93c56cd4f95
2024-12-16 19:49:46 - root - INFO - Created new conversation in MongoDB
2024-12-16 19:49:46 - root - INFO - Conversation stored in MongoDB
